In [20]:
#!pip install plotly

In [87]:
import os
import pandas as pd
import plotly.express as px

In [88]:
data_dir = '../data'
predictions_path = os.path.join(data_dir, 'predictions.tsv')
tissues_path = os.path.join(data_dir, 'tissues_cell_types.tsv')

In [89]:
predictions = pd.read_csv(predictions_path, sep='\t')
tissues = pd.read_csv(tissues_path, sep='\t')

In [90]:
predictions.head()

,taxid1,taxid1_label,source,source_name,taxid2,taxid2_label,target,target_name,experimental_evidence_score,databases_evidence_score,average_score,group1,group2,edge_type
0,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
1,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000475084,DUOX2,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
2,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000362057,NOX1,0.000,0.77,0.385,KOG0039,KOG0033,inter-species
3,5741,Giardia intestinalis,5741.EDO78093,RabF,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.994,0.00,0.497,KOG0078,KOG0033,inter-species
4,5741,Giardia intestinalis,5741.EDO78093,RabF,9606,Homo sapiens,9606.ENSP00000312734,RAB8B,0.994,0.00,0.497,KOG0078,KOG0033,inter-species


In [91]:
tissues.head()

,Gene,Tissue,Gene name,Cluster,Cell type,Read count,pTPM
0,9606.ENSP00000000233,brain,ARF5,c-9,excitatory neurons,48578715.0,104.5
1,9606.ENSP00000000233,brain,ARF5,c-26,inhibitory neurons,14766772.0,91.4
2,9606.ENSP00000000233,brain,ARF5,c-43,microglial cells,390440.0,76.8
3,9606.ENSP00000000233,brain,ARF5,c-35,astrocytes,1538564.0,56.5
4,9606.ENSP00000000233,brain,ARF5,c-12,oligodendrocytes,8026745.0,52.9


In [92]:
aux = pd.merge(predictions.rename({'target':'Gene'}, axis=1), tissues, on='Gene', how='left')
aux['Cell type'] = aux['Cell type'].fillna("Not available")

In [93]:
counts_tissues = aux.groupby(['taxid1', 'Tissue']).count()['taxid2'].reset_index()
counts_tissues = counts_tissues.rename({'taxid2':'edges_tissue'}, axis=1)
counts_tissues.head()

,taxid1,Tissue,edges_tissue
0,5664,bladder,20
1,5664,blood,79
2,5664,bone marrow,52
3,5664,brain,7258
4,5664,eye,523


In [94]:
counts_cells = aux.groupby(['taxid1', 'Tissue', 'Cell type']).count()['taxid2'].reset_index()
counts_cells = counts_cells.rename({'taxid2':'edges_cell_type'}, axis=1)
counts_cells.head()

,taxid1,Tissue,Cell type,edges_cell_type
0,5664,bladder,Not available,20
1,5664,blood,Not available,79
2,5664,bone marrow,b-cells,13
3,5664,bone marrow,erythroid cells,13
4,5664,bone marrow,macrophages,13


In [95]:
aux = pd.merge(aux, counts_tissues, on=['taxid1', 'Tissue'], how='left')
aux = pd.merge(aux, counts_cells, on=['taxid1', 'Tissue', 'Cell type'], how='left')
aux.head()

,taxid1,taxid1_label,source,source_name,taxid2,taxid2_label,Gene,target_name,experimental_evidence_score,databases_evidence_score,...,group2,edge_type,Tissue,Gene name,Cluster,Cell type,Read count,pTPM,edges_tissue,edges_cell_type
0,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-42,oligodendrocyte precursor cells,1265946.0,703.8,9994,1647
1,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-21,inhibitory neurons,20255965.0,296.3,9994,1678
2,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-35,astrocytes,1538564.0,202.8,9994,1641
3,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-4,excitatory neurons,94683635.0,167.6,9994,1677
4,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-43,microglial cells,390440.0,163.9,9994,1648


In [96]:
aux.sort_values(by="edges_tissue", ascending=False)

,taxid1,taxid1_label,source,source_name,taxid2,taxid2_label,Gene,target_name,experimental_evidence_score,databases_evidence_score,...,group2,edge_type,Tissue,Gene name,Cluster,Cell type,Read count,pTPM,edges_tissue,edges_cell_type
438418,7209,Loa loa,7209.EJD74571.1,EJD74571.1,9606,Homo sapiens,9606.ENSP00000242786,CD97,0.000,0.899,...,KOG4193,inter-species,brain,ADGRE5,c-43,microglial cells,390440.0,17.9,34095,5573
325385,7209,Loa loa,7209.EFO27577.1,EFO27577.1,9606,Homo sapiens,9606.ENSP00000343027,GNAI1,0.000,0.898,...,KOG0082,inter-species,brain,GNAI1,c-12,oligodendrocytes,8026745.0,593.6,34095,5727
325338,7209,Loa loa,7209.EFO27577.1,EFO27577.1,9606,Homo sapiens,9606.ENSP00000400717,GNA13,0.000,0.898,...,KOG0082,inter-species,brain,GNA13,c-35,astrocytes,1538564.0,137.1,34095,5544
325339,7209,Loa loa,7209.EFO27577.1,EFO27577.1,9606,Homo sapiens,9606.ENSP00000400717,GNA13,0.000,0.898,...,KOG0082,inter-species,brain,GNA13,c-43,microglial cells,390440.0,120.4,34095,5573
325340,7209,Loa loa,7209.EFO27577.1,EFO27577.1,9606,Homo sapiens,9606.ENSP00000400717,GNA13,0.000,0.898,...,KOG0082,inter-species,brain,GNA13,c-39,excitatory neurons,7483738.0,59.2,34095,5771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248333,5865,Babesia bovis,5865.XP_001611086.1,BBOV_IV011660,9606,Homo sapiens,9606.ENSP00000379350,ITGB1,0.000,0.899,...,KOG1226,inter-species,muscle,NaN,NaN,Not available,NaN,NaN,3,3
49845,5865,Babesia bovis,5865.XP_001611807.1,17.m07600,9606,Homo sapiens,9606.ENSP00000337838,RTN4,0.786,0.000,...,KOG1792,inter-species,muscle,NaN,NaN,Not available,NaN,NaN,3,3
1087,5865,Babesia bovis,5865.XP_001608744.1,16.m00728,9606,Homo sapiens,9606.ENSP00000246533,CAPNS1,0.000,0.719,...,KOG0037,inter-species,muscle,NaN,NaN,Not available,NaN,NaN,3,3
51458,5865,Babesia bovis,5865.XP_001611807.1,17.m07600,9606,Homo sapiens,9606.ENSP00000211998,VCL,0.000,0.899,...,KOG3681,inter-species,spinal cord,NaN,NaN,Not available,NaN,NaN,2,2


In [97]:
aux.head()

,taxid1,taxid1_label,source,source_name,taxid2,taxid2_label,Gene,target_name,experimental_evidence_score,databases_evidence_score,...,group2,edge_type,Tissue,Gene name,Cluster,Cell type,Read count,pTPM,edges_tissue,edges_cell_type
0,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-42,oligodendrocyte precursor cells,1265946.0,703.8,9994,1647
1,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-21,inhibitory neurons,20255965.0,296.3,9994,1678
2,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-35,astrocytes,1538564.0,202.8,9994,1641
3,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-4,excitatory neurons,94683635.0,167.6,9994,1677
4,5722,Trichomonas vaginalis,5722.XP_001305090.1,XP_001305090.1,9606,Homo sapiens,9606.ENSP00000367408,CASK,0.0,0.77,...,KOG0033,inter-species,brain,CASK,c-43,microglial cells,390440.0,163.9,9994,1648


In [98]:
aux['Tissue'].unique().tolist()

['brain',
 'skin',
 'gastrointestinal tract',
 'intestine',
 'urogenital system',
 'muscle',
 'lung',
 'kidney',
 'placenta',
 'mouth',
 'blood',
 'bladder',
 'eye',
 'heart',
 'liver',
 'spleen',
 'bone marrow',
 'spinal cord']

In [99]:
fig = px.icicle(aux, path=[px.Constant("Parasites"), 'taxid1_label', 'Tissue', 'Cell type'], values='edges_cell_type',
                  color='edges_cell_type', hover_data=['edges_tissue', 'edges_cell_type', 'taxid1', 'taxid1_label', 'pTPM'],
                  color_continuous_scale='Burgyl', height=900, width=1200)
fig.show()